In [1]:
# Final Project: Implement the histogram equalization
import IP
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
from matplotlib.image import imsave
from scipy.signal import convolve2d

In [2]:
dLevel = np.arange(257)
house = IP.imread("house.jpeg")[:,:,0]
lena = IP.imread("../lena.jpg")
forest = IP.imread("forest.jpg")
f16 = IP.imread("F16.png")[:,:,0]
bottle = IP.imread("bottle.png")[:,:,0]
einstein = IP.imread("Einstein.png")[:,:,0]
cabin = IP.imread("Cabin.png")[:,:,0]
hands = IP.imread("Hands.png")[:,:,0]
party = IP.imread("party.png")[:,:,0]
images = {
    "house":house,
    "lena":lena,
    "forest":forest,
    "f16":f16,
    "bottle":bottle,
    "einstein":einstein,
    "cabin":cabin,
    "hands":hands,
    "party":party
}

# General Histogram equalization

In [12]:
def NormHist(image,density=True):
    hist,bin_edges = np.histogram(a=image,bins=dLevel,density=density)
    return hist


def showHistogram(image,bins=dLevel):
    """
    image:input image, np.array
    bins:specify the edge of histogram
    show image's histogram
    """
    # flatten the image when passed into the histeq function
    norm_hist = NormHist(image)
    plt.xlabel("Gray level")
    plt.ylabel("density")
    plt.plot(norm_hist)

def showCDF(image,bins=dLevel):
    """
    image: input image np.array
    show the cumulative distribution function(CDF) of an image
    """
    hist,bin_edges = np.histogram(a=image.ravel(),bins=bins)
    width,height = image.shape
    plt.xlabel("Gray level")
    hist=hist/float(width*height)
    cdf = IP.cumsum(hist)
    plt.plot(cdf)

    
def showPDF(image,bins=dLevel):
    hist,bin_edges = np.histogram(a=image.ravel(),bins=bins)
    plt.xlabel("Gray level")
    plt.ylabel("density")
    width,height = image.shape
    hist=hist/float(width*height)
    plt.plot(hist)


def contrast(image):
    """
    calculate the contrast balance of an image
    Link: 
    :return: The contrast value of an image
    """
    return image.std()


def HistogramFlatness(hist):
    nonzero_intensity_num = np.where(hist > 0)[0].shape[0]
    hist = hist-np.mean(hist)
    hist = np.square(hist)
    return hist.sum()/float(nonzero_intensity_num)


def discreteEntropy(image):
    hist,bin_edges = np.histogram(a=image,bins=dLevel)
    width,height = image.shape
    hist = hist/float(width*height)
    hist = hist[np.nonzero(hist)]
    return -np.multiply(hist,np.log2(hist)).sum()


def ContrastPerPixel(image):
    """
    calculate the root-mean-square of an image to evaluate it's contrast
    :image Original image to be calulated
    :return: The rms of the image to evaluate it's contrast balance
    """
    kernel = [-1, -1, -1, -1, 8, -1, -1, -1,-1]
    kernel = np.array(kernel,dtype=float)
    kernel = np.reshape(kernel,(3,3))
    kernel/=float(8)
    diff_image = convolve2d(image,kernel,mode="same")
    return np.mean(np.abs(diff_image))

def showHistogramAndCDF(image):
    """
    Show the histogram of an image, including it's hist, pdf,cdf
    :image Original image
    :return: None
    """
    IP.subplot(2,1,1);IP.title("histogram")
    showHistogram(image,bins=dLevel)
    IP.subplot(2,1,2);IP.title("cdf function")
    showCDF(image,bins=dLevel)
    IP.plt.subplots_adjust(hspace=1)

def printTotalMessage(imageName,func):
    image = images[imageName]
    print(str(func.func_name)+" of "+imageName+" :")
    print("Original:%f"%func(image))
    print("HE:%f"%func(HE(image)))
    print("BBHE:%f"%func(BBHE(image,mode="mean")))
    
def compareGraph(imagename,show_func,bins=dLevel,mode="mean"):
    image = images[imagename]
    IP.subplot(3,1,1);plt.title("Original "+imagename);show_func(image)
    image_HE = IP.histeq(image)
    image_HE*=255
    image_BBHE = BBHE(image,mode=mode)
    IP.subplot(3,1,2);plt.title("HE");show_func(image_HE)
    IP.subplot(3,1,3);plt.title("BBHE");show_func(image_BBHE)
    IP.plt.subplots_adjust(hspace=1.2)
    plt.savefig("Total_compare_"+imagename+"_"+show_func.func_name+".png")

    
def SplitHistogramByThreshold(image,th):
    """
    return th,lower part histgram,higher part histogram
    """
    assert type(th)==int
    hist = NormHist(image)
    return th,hist[:th],hist[th:]

def CompareHEBBHEHistogram(imageName,**kwargs):
    image = images[imageName]
    th = 0
    if "th_func" in kwargs.keys():
        th = int(kwargs["th_func"](image))
    if "th" in kwargs.keys():
        th = kwargs["th"]
    BBHE_image,HE_image = BBHE(image,mode="mean"),HE(image)
    BBHE_th,BBHE_lower,BBHE_higher = SplitHistogramByThreshold(BBHE_image,th)
    HE_th,HE_lower,HE_higher = SplitHistogramByThreshold(HE_image,th)
    plt.figure(figsize=(10,8))
    IP.subplot(2,2,1);plt.title("HE %s lower hist"%imageName)
  
    plt.xlabel("Gray level");plt.ylabel("Density")
    plt.plot(HE_lower);

    IP.subplot(2,2,2);plt.title("HE %s higher hist"%imageName)
    plt.xlabel("Gray level");plt.ylabel("Density")
    plt.plot(np.arange(th,256),HE_higher);

    IP.subplot(2,2,3);plt.title("BBHE %s lower hist"%imageName)
    plt.xlabel("Gray level");plt.ylabel("Density")
    plt.plot(BBHE_lower);

    IP.subplot(2,2,4);plt.title("BBHE %s higher hist"%imageName)
    plt.xlabel("Gray level");plt.ylabel("Density")
    plt.plot(np.arange(th,256),BBHE_higher);

    plt.subplots_adjust(hspace=0.3,wspace=0.3)
    plt.savefig("%s_BBHE_HE_Hist_compare.png"%imageName)
    
    print("Histogram Flatness of lower HE:%f"%HistogramFlatness(HE_lower))
    print("Histogram Flatness of higher HE:%f"%HistogramFlatness(HE_higher))
    print("Histogram Flatness of lower BBHE:%f"%HistogramFlatness(BBHE_lower))
    print("Histogram Flatness of higher BBHE:%f"%HistogramFlatness(BBHE_higher))

# Brightness Preserving Bi-Histogram Equalization

In [4]:
backup = {"mean":np.mean,"median":np.median}
def BBHE(image,mode="mean",L=256):
    """
    :image The original image to be processed
    :dLevel Maximum Gray level(default as 256)
    :mode Use which threshold value(default as mean to use average value, could also use median value)
    :return: The image after BBHE with specified threshold value
    """
    
    th = int(backup[mode](image))
    histogram = np.histogram(image,dLevel)[0] # Get the global histogram of the image
    h,w = image.shape;assert h*w == histogram.sum() # assert we get the right histogram
    locdf,hicdf = splitHistogram(histogram,th)
    result = np.zeros(image.shape)
    locdfmax,hicdfmax = locdf.shape[0]-1,hicdf.shape[0]-1
    X_0,X_m,X_m_1,X_L_1 = 0,th,th+1,255
    for i in range(h):
        for v in range(w):
            value = image[i,v]
            if value<=th:
                result[i,v] = X_0 + (X_m-X_0)*locdf[value]
            else:
                result[i,v] = X_m_1 + (X_L_1-X_m_1)*hicdf[value-X_m_1]
    return result
    
def splitHistogram(his,th):
    """
    Helper function to calculate the image after BBHE
    Calculate two cumulative function based on the specified threshold value and a histogram
    """
    lohis,hihis = his[:th+1],his[th+1:]
    lohis = np.array(lohis,dtype=float)
    hihis = np.array(hihis,dtype=float)
    lohis/=float(lohis.sum())
    hihis/=float(hihis.sum())
    locdf,hicdf = np.cumsum(lohis),np.cumsum(hihis)
    return locdf,hicdf
def HE(image):
    """
    Show the comparison between standard Hist image and BBHE image
    :image Original Image
    :return: image after Standard hist and BBHE
    """
    stanardHist = IP.histeq(image)
    return stanardHist*255